# Amazon Bedrock for Marketing Campaign

Rather than relying on generic, one-size-fits-all product recommendations, Claude in Amazon Bedrock can tailor the recommendations to each customer's unique interests and preferences. This level of personalization helps to make the email content more valuable and interesting to the recipients. Additionally, Claude can dynamically update the recommendations based on new customer data, ensuring the emails remain fresh and relevant over time.


In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd
from io import StringIO
import random

%store -r


***Setting up Claude for Personalized Product Recommendations***

Here, we setup integration with Amazon Bedrock's Claude AI assistant to generate personalized marketing email content with product recommendations. The generate_personalize_simple_prompt function takes a list of products and parameters for the Claude model, and returns a prompt that can be used to generate a marketing email with personalized product recommendations.

In [ ]:
bedrock = boto3.client('bedrock-runtime')

# Model parameters
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
max_tokens = 1000



def generate_personalize_simple_prompt(product_list, model_id, max_tokens_to_sample=1000):
    prompt_template = f'''
Human: Write a marketing email advertising several products available in our store, given the product recommendations below. The products to recommend and their information is contained in the <product> tag. Put the email between <email> tags.

<product>
{product_list}
</product>

Assistant: Email body:
<email>.
Assistant:
'''
    if 'claude' in model_id:
        prompt_input = json.dumps({"prompt": prompt_template, "max_tokens": max_tokens_to_sample, "anthropic_version": "bedrock-2023-05-31"})
    
    return prompt_input



***Personalized Email Generation***

We first group the user recommendations by user ID, creating a DataFrame that associates each user with their recommended items. As we iterate through each user, we use the generated prompt which is populated with recommended items. This is then passed Claude AI model to produce personalized marketing email content for that user. The generated email content is then printed for each user.

In [ ]:
def getPersonalizedEmail(bedrock_client, model_id, max_tokens_to_sample, prompt ):
    
    personalized_email = "ERROR"
    
    body = json.dumps({
      "max_tokens": max_tokens_to_sample,
      "messages": [{"role": "user", "content": prompt}],
      "anthropic_version": "bedrock-2023-05-31"
    })
    
    response = bedrock.invoke_model(body=body, modelId=model_id)
    
    response_body = json.loads(response.get('body').read())  
    personalized_email = response_body['content'][0]['text'].strip()

    
    return personalized_email

### Generate Email Template for Users

We generate personalized email templates for users based on their unique recommendations. It first groups recommendations by user ID and username, then creates customized email content for each user. Additionally, we can selects a random user to demonstrate a sample personalized email.

In [ ]:
user_recommendations = recommendations_df.groupby(['USER_ID', 'Username'])['Recommended_Items'].apply(list).reset_index()
max_tokens_to_sample = 10000


random_row = user_recommendations.sample(1)

user_id = random_row['USER_ID'].values[0]
username = random_row['Username'].values[0]
recommendations = random_row['Recommended_Items'].values[0]

# Generate the personalized email content
prompt_input_json = generate_personalize_simple_prompt(recommendations, model_id, max_tokens_to_sample)
personalized_email = getPersonalizedEmail(bedrock, model_id, max_tokens_to_sample, prompt_input_json)

print(f"User ID: {user_id}\nUsername: {username}\nEmail Content:\n{personalized_email}\n")

In [ ]:
emails = []
       
    
for index, row in user_recommendations.iterrows():
    user_id = row['USER_ID']
    username = row['Username']
    recommendations = row['Recommended_Items']
        
    prompt_input_json = generate_personalize_simple_prompt(recommendations, model_id, max_tokens_to_sample)
    personalized_email = getPersonalizedEmail(bedrock, model_id, max_tokens_to_sample, prompt_input_json)  
    
    print(f"User ID: {user_id}\nUsername: {username}\nEmail Content:\n{personalized_email}\n")

     

In [ ]:
%store personalized_email